## LDA(Latent Dirichlet Allocation), 잠재디리클레할당
    - 토픽 모델링에 사용되는 대표적인 알고리즘
    
- 모듈
    - gensim.models.wrappers.LdaMallet 모듈사용 => 깁스 샘플링을 사용하여 훈련 코퍼스에서 LDA 모델 추천가능
        - 깁스 샘플링 => 2번째로 생성될 표본은 1차 생성 표본의 영향을 받으며, 나머지 변수는 가만히 두고 한 변수에만 변화를 준다.
            - ex) p(x1, y1, z1)가 존재할 경우, 변수명+표본의 차수
            - 변경 순서 => p(y1, z1)을 유지시킨 채 x1 -> x2 로 변경 => p(x2, z1)을 유지시킨 채 y1 -> y2 로 변경 => ...
            - 1차 변경 결과 => p(x2, y2, z2)

        - 검색창에 mallet.cs.umass.edu/dist/mallet-2.0.8.zip 치면 해당 zip파일 자동다운로드
        
        
- CoherenceMetric
	- 토픽 내 일관성 지수를 계산, 최적의 토픽 개수를 찾기위해서 사용
	- coherence = 'c_v' 사용
	- score가 높은 토픽 개수 사용
    

- Corpora 라이브러리
	- from gensim import corpora / from gensim.corpora.dictionary import Dictionary
	- 텍스트 데이터의 특성 상 수치화해서 분석 및 연산 진행해야함.
	- 텍스트를 가지고 분석이나 연산에 용이하게 데이터 집합을 만들어 주는 라이브러리
	- document = ['a', 'b', 'c']라는 리스트가 존재할 때
	- dict = corpora.Dictionary(document) / dict = Dictionary(document) 
	- dict는 리스트가 사전형식으로 변환 (ex) {'a': 0, 'b': 1, 'c': 2}
	- doc2bow를 통해서 단어를 숫자로 변경하고 카운트까지 알 수 있음
	- dict.doc2bow(input) 으로 input에 있는 텍스트 사전을 통해 단어별 개수 확인
	- ex) [(0, 1), (1, 5)]이면 0번에 위치한 단어는 1번, 1번에 위치한 단어는 5번 나왔다는 의미

In [1]:
# !pip install konlpy
# 3.8.5 버전이후로 gensim.models.wrappers.LdaMallet 삭제 => 3.8.3 버전 사용
# !pip install gensim==3.8.3

In [2]:
from tqdm.notebook import tqdm # 진행률 프로세스바
import itertools # 효율적인 루핑을 위한 이터레이터
from gensim.models.ldamodel import LdaModel # LDA 모델
from gensim.models.callbacks import CoherenceMetric 
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel # 최적의 토픽개수 구하는 모델
import pandas as pd
import numpy as np
from konlpy.tag import Okt
import os

In [3]:
# 전체 데이터 읽기
# df = pd.read_parquet('after_preprocessing.parquet')

# 필요한 컬럼만 읽고 싶을 경우
df = pd.read_parquet('after_preprocessing.parquet', columns=['source', 'content'])

In [10]:
# Okt로 토큰화
def tokenizedOkt(tourDf):
    okt = Okt()
    tour_tokenized = [] 

    # 해당 관광지에 관한 content들 토큰화
    tqdmDf = tqdm(tourDf)

    for tourInfo in tqdmDf:
        tqdmDf.set_description(f'Processing tokenized')
        tokenized_doc = okt.pos(tourInfo)
        
        # 명사만 사용
        tokenized_list = [w for w,t in tokenized_doc if t =='Noun']
        tour_tokenized.append(tokenized_list)
        
    return tour_tokenized

In [5]:
# 토크나이징한 데이터를 입력으로 받아서, 이 데이터들을 바탕으로 사전 제작 
# corpus에는 (token_id, token_count)가 튜플형태로 저장
def dictionary(data_word):
    
    # 토큰화된 문장리스트로 사전구성
    id2word=corpora.Dictionary(data_word)
    id2word.filter_extremes(no_below = 20) # 20회 이하로 등장한 단어는 삭제
    texts = data_word
    # 토큰화된 문장리스트의 각 단어를 통해서 해당 단어가 총 몇번 등장하는 지 계산
    corpus=[id2word.doc2bow(text) for text in texts]
    return id2word, texts, corpus

In [6]:
# 최적의 토픽 개수 도출
def compute_coherence_values(dictionary, corpus, texts, start, limit, step):
    
    os.environ.update({'MALLET_HOME':r'C:/pythonPrj/jejuModel/mallet-2.0.8/'})
    
    # 설치한 mallet 경로
    mallet_path = 'C:/pythonPrj/jejuModel/mallet-2.0.8/bin/mallet'
    
    # 토픽 개수에 따른 모델 저장
    model_list = []
    
    # 모델별 일관성 지수 저장
    coherence_values = []
    
    print(f'start: {start}, limit: {limit}, step: {step}')

    # 진행상황 확인 변수 선언
    tqdmSet = tqdm(range(start, limit, step))
    
    for num_topics in tqdmSet:
        # 진행 상황 확인(현재 진행되는 num_topics로 해당 문자열 변경)
        tqdmSet.set_description(f'Processing num_topics')
        
        # 토픽 개수별 모델 
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        
        # 모델별 토픽의 일관성 지수 계산 모델
        coherencemodel = CoherenceModel(model=model, texts=data_word, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
    return model_list, coherence_values

In [7]:
def LDA(data_word, Data_list, start, limit, step):
    id2word, texts, corpus = dictionary(data_word)
    
    print("Create LDA instance")
    os.environ.update({'MALLET_HOME':r'C:/pythonPrj/jejuModel/mallet-2.0.8/'})
    
    # 설치한 mallet 경로
    mallet_path = 'C:/pythonPrj/jejuModel/mallet-2.0.8/bin/mallet'
    
#     # LDA 모델
#     ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
    
#     # 토픽의 일관성 지수 계산 
#     coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
#     coherence_ldamallet = coherence_model_ldamallet.get_coherence()
    
    # 해당 반복문 사용 불가할 시 위의 코드 사용
    # 위의 compute_coherence_values를 통한 최적의 토픽 개수 사용
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=start, limit=limit, step=step)
    
    x = range(start, limit, step)
    topic_num = 0
    count = 0
    max_coherence = 0
    for m, cv in zip(x, coherence_values):
        # score가 높을수록 좋음
        print("Num Topics =", m, " has Coherence Value of", cv)
        coherence = cv
        if coherence >= max_coherence:
            max_coherence = coherence
            topic_num = m
            model_list_num = count   
        count = count+1
        
    # 최적의 모델 도출
    optimal_model = model_list[model_list_num]
    model_topics = optimal_model.show_topics(formatted=False, num_words=10)
    return optimal_model, model_topics

### 한 관광지에 대한 토픽 모델링 진행
    - 한 관광지에 대한 다양한 특성별 토픽을 통하여 특성별로 상위에 위치한 단어를 키워드로 사용

In [11]:
# type(tourDf) => pandas.core.series.Series(원본 데이터)
tourDf = df[df.source == '공천포']['content'] 

# seires => DataFrame으로 변환 (원본 데이터)
# tour_OriDf = pd.DataFrame(tourDf)

# 토큰화 진행
tour_tokenized = tokenizedOkt(tourDf)

data_word = tour_tokenized   # 토큰화 데이터
Data_list = tourDf             # 원본 데이터
start=5; limit=10; step=1;          # 모델에서 평가해볼 토픽의 개수 시작, 끝, 증분

  0%|          | 0/347 [00:00<?, ?it/s]

In [12]:
model, topics = LDA(data_word, Data_list, start, limit, step)

Create LDA instance
start: 5, limit: 10, step: 1


  0%|          | 0/5 [00:00<?, ?it/s]

Num Topics = 5  has Coherence Value of 0.45843252051599437
Num Topics = 6  has Coherence Value of 0.44210563238726636
Num Topics = 7  has Coherence Value of 0.4644892361686814
Num Topics = 8  has Coherence Value of 0.4667237867115923
Num Topics = 9  has Coherence Value of 0.4742658558790529


In [13]:
# 결과확인 (공천포)
topics

[(0,
  [('전복', 0.17200217037438958),
   ('한치', 0.14378730330982095),
   ('된장', 0.060499186109603906),
   ('전복죽', 0.05344546934346175),
   ('맛집', 0.04584915897992404),
   ('베이스', 0.0341833966359197),
   ('고등어', 0.030656538252848618),
   ('반찬', 0.030113944655453067),
   ('국물', 0.025501899077590883),
   ('냉동', 0.025230602278893108)]),
 (1,
  [('우리', 0.06780798925814031),
   ('근처', 0.058744545149378984),
   ('숙소', 0.04296743873783149),
   ('미리', 0.03759650889560255),
   ('도착', 0.03591809331990601),
   ('오늘', 0.03491104397448808),
   ('다음', 0.02853306478684122),
   ('날씨', 0.02853306478684122),
   ('점심', 0.027861698556562606),
   ('이제', 0.027861698556562606)]),
 (2,
  [('정말', 0.09620596205962059),
   ('추천', 0.06063685636856368),
   ('느낌', 0.05521680216802168),
   ('진짜', 0.0535230352303523),
   ('조금', 0.053184281842818426),
   ('생각', 0.049796747967479675),
   ('그냥', 0.03353658536585366),
   ('사실', 0.03116531165311653),
   ('여기', 0.030149051490514906),
   ('저희', 0.030149051490514906)]),
 (3,
 

#### 결과 : 일부 토픽의 단어는 키워드로 사용할만 하지만, 전체적으로 토픽 전체가 의미를 가진다고 보기는 힘들다. 전체 관광지를 대상으로 해당 과정을 진행한다고 가정하였을 때, 수작업을 통한 토픽 분류가 아니면 데이터의 가공이 힘들 것 같다.

### 여러 관광지에 대한 토픽 모델링 진행
    - 관광지수와 output 토픽의 개수를 일치시켜, 해당 관광지에 대한 토픽 도출 예상
    - 관광지 별 토픽으로 제대로 도출된다면 해당 토픽의 단어는 관광지의 키워드로 사용가능하다고 판단
    - (공천포, 휴애리 자연생활공원, 성산일출봉(UNESCO 세계자연유산, 본태박물관, 제주올레 14코스) 5개의 관광지로 진행

In [16]:
tourDf = df[(df.source == '공천포') | (df.source == '휴애리 자연생활공원') | (df.source == '성산일출봉(UNESCO 세계자연유산)') | (df.source == '본태박물관') | (df.source == '제주올레 14코스')]['content']

# 토큰화 진행
tour_tokenized = tokenizedOkt(tourDf)

data_word = tour_tokenized   # 토큰화 데이터
Data_list = tourDf             # 원본 데이터
# 관광지가 5개 이기때문에 토픽의 개수도 5개로 고정
start=5; limit=6; step=1;          # 모델에서 평가해볼 토픽의 개수 시작, 끝, 증분

  0%|          | 0/2382 [00:00<?, ?it/s]

In [17]:
model, topics = LDA(data_word, Data_list, start, limit, step)

Create LDA instance
start: 5, limit: 6, step: 1


  0%|          | 0/1 [00:00<?, ?it/s]

Num Topics = 5  has Coherence Value of 0.7202211203538502


In [18]:
# 결과확인 (공천포, 휴애리 자연생활공원, 성산일출봉(UNESCO 세계자연유산, 본태박물관, 제주올레 14코스)
topics

[(0,
  [('애리', 0.05429989451294378),
   ('공원', 0.0380358655991146),
   ('수국', 0.03699828799695644),
   ('생활', 0.029916820862226988),
   ('축제', 0.02547253013298286),
   ('돼지', 0.023129334048109015),
   ('체험', 0.02269701004720978),
   ('동백꽃', 0.015252390751724973),
   ('포토', 0.013540387708164006),
   ('핑크', 0.01345392290798416)]),
 (1,
  [('박물관', 0.044302260513340304),
   ('작품', 0.033089438619830436),
   ('태', 0.027915914394011383),
   ('전시', 0.019870556312227467),
   ('공간', 0.019300412826116796),
   ('전시관', 0.014992662042169501),
   ('관람', 0.014834288851583204),
   ('안도', 0.014095213962180482),
   ('건물', 0.010442072365989884),
   ('전통', 0.009185645054005258)]),
 (2,
  [('코스', 0.04929985417790296),
   ('올레', 0.029653822336749167),
   ('마을', 0.01534128363396656),
   ('시작', 0.013123988733744832),
   ('오름', 0.011935438764707058),
   ('올레길', 0.011406084576816284),
   ('바람', 0.00955833882663151),
   ('바다', 0.008829228341423464),
   ('오늘', 0.008719362377898963),
   ('길이', 0.00862947204410619)]

#### 결과 : 5개의 토픽이 관광지 별로 잘 나누어져 있음을 토픽내부의 단어를 통해서 알 수 있다. 하지만 전체 관광지를 대상으로 진행한다고 생각하면, 토픽을 미리 지정할 수 없기에 이 후에 각 토픽 별로 라벨링을 할 필요성이 있다.

### 비슷한 특성을 가진 관광지들에 대한 토픽 모델링 진행
    - 관광지수와 output 토픽의 개수를 일치시켜, 해당 관광지에 대한 토픽 도출 예상
    - 관광지 별 토픽으로 제대로 도출된다면 해당 토픽의 단어는 관광지의 키워드로 사용가능하다고 판단
    - (바이나흐튼 크리스마스 박물관, 세계자동차 & 피아노박물관, 한국야구명예전당(야구박물관), 본태박물관, 초콜릿박물관) 5개의 관광지로 진행

In [19]:
tourDf = df[(df.source == '바이나흐튼 크리스마스 박물관') | (df.source == '세계자동차 & 피아노박물관') | (df.source == '한국야구명예전당(야구박물관)') | (df.source == '본태박물관') | (df.source == '초콜릿박물관')]['content']

# 토큰화 진행
tour_tokenized = tokenizedOkt(tourDf)

data_word = tour_tokenized   # 토큰화 데이터
Data_list = tourDf             # 원본 데이터
# 관광지가 5개 이기때문에 토픽의 개수도 5개로 고정
start=5; limit=6; step=1;          # 모델에서 평가해볼 토픽의 개수 시작, 끝, 증분

  0%|          | 0/2905 [00:00<?, ?it/s]

In [20]:
model, topics = LDA(data_word, Data_list, start, limit, step)

Create LDA instance
start: 5, limit: 6, step: 1


  0%|          | 0/1 [00:00<?, ?it/s]

Num Topics = 5  has Coherence Value of 0.6590202565322094


In [21]:
# (바이나흐튼 크리스마스 박물관, 세계자동차 & 피아노박물관, 한국야구명예전당(야구박물관), 본태박물관, 초콜릿박물관)
topics

[(0,
  [('크리스마스', 0.0861721644663477),
   ('바이', 0.021979207885175725),
   ('흐', 0.021394654483974245),
   ('마켓', 0.016619302852620598),
   ('구경', 0.015450196050217633),
   ('진짜', 0.015423216662469872),
   ('소품', 0.014110219792078852),
   ('여기', 0.013741501492859456),
   ('분위기', 0.012842188567934098),
   ('정말', 0.01254541530270873)]),
 (1,
  [('작품', 0.03272612569986594),
   ('태', 0.023315721683358305),
   ('공간', 0.020914929597210176),
   ('관람', 0.01605201132052327),
   ('전시관', 0.015946867141567875),
   ('안도', 0.011671003864048577),
   ('느낌', 0.008989827300686065),
   ('건물', 0.008472868420822052),
   ('전통', 0.008008481630435734),
   ('거울', 0.0077806692426990514)]),
 (2,
  [('카페', 0.014022191400832178),
   ('우리', 0.01217753120665742),
   ('숙소', 0.008890429958391123),
   ('코스', 0.0075034674063800275),
   ('제주시', 0.007052704576976421),
   ('바로', 0.0068793342579750345),
   ('호텔', 0.006782246879334258),
   ('그냥', 0.006761442441054091),
   ('다음', 0.006712898751733703),
   ('바다', 0.00661581137

#### 결과 : 단어의 분포를 통해 세계자동차 & 피아노 박물관과 초콜릿 박물관의 경우 같은 토픽에 분류되었음을 볼 수 있다. 

### 비슷한 특성을 가진 관광지들에 대한 토픽 모델링 진행(제주 올레길)
    - 관광지수와 output 토픽의 개수를 일치시켜, 해당 관광지에 대한 토픽 도출 예상
    - 관광지 별 토픽으로 제대로 도출된다면 해당 토픽의 단어는 관광지의 키워드로 사용가능하다고 판단
    - 제주 올레길의 경우에도 코스별로 관광지가 나누어져 있는데 박물관보다 해당 관광지들 간의 특성이 유사할 것으로 예상된다.
    - (제주올레 3코스, 제주올레 4코스, 제주올레 20코스, 제주올레 7-1코스, 제주올레 10코스) 5개의 관광지로 진행

In [22]:
tourDf = df[(df.source == '제주올레 3코스') | (df.source == '제주올레 4코스') | (df.source == '제주올레 20코스') | (df.source == '제주올레 7-1코스') | (df.source == '제주올레 10코스')]['content']

# 토큰화 진행
tour_tokenized = tokenizedOkt(tourDf)

data_word = tour_tokenized   # 토큰화 데이터
Data_list = tourDf             # 원본 데이터
# 관광지가 5개 이기때문에 토픽의 개수도 5개로 고정
start=5; limit=6; step=1;          # 모델에서 평가해볼 토픽의 개수 시작, 끝, 증분

  0%|          | 0/2475 [00:00<?, ?it/s]

In [23]:
model, topics = LDA(data_word, Data_list, start, limit, step)

Create LDA instance
start: 5, limit: 6, step: 1


  0%|          | 0/1 [00:00<?, ?it/s]

Num Topics = 5  has Coherence Value of 0.41673649033993215


In [24]:
# (제주올레 3코스, 제주올레 4코스, 제주올레 20코스, 제주올레 7-1코스, 제주올레 10코스)
topics

[(0,
  [('우리', 0.012329850062114277),
   ('마음', 0.010987915949750546),
   ('송악산', 0.010569391944409383),
   ('때문', 0.009466614406526318),
   ('산방산', 0.00801838848328229),
   ('여행', 0.007858950766961848),
   ('지금', 0.0073739943798205),
   ('하나', 0.00711490809079978),
   ('위해', 0.006251287127397379),
   ('이야기', 0.006111779125616991)]),
 (1,
  [('스탬프', 0.021345877716297486),
   ('오늘', 0.015536290395256204),
   ('버스', 0.014346736838310835),
   ('중간', 0.01327734020630944),
   ('숙소', 0.012051739571880877),
   ('완주', 0.011468978485902588),
   ('식당', 0.011180601866037044),
   ('아침', 0.009258091066933415),
   ('출발', 0.008044506124999249),
   ('종점', 0.007882294276324881)]),
 (2,
  [('오름', 0.02924475256578988),
   ('해변', 0.020445993289710272),
   ('봉', 0.014557497791041825),
   ('해녀', 0.014440098615307618),
   ('바람', 0.013278464665937555),
   ('포구', 0.013222854530063458),
   ('해수욕장', 0.010596820336008799),
   ('해안', 0.010442347736358524),
   ('풍경', 0.008187047781464524),
   ('김녕', 0.0076741987506

#### 결과 : 관광지 별로 토픽이 제대로 ㅂ